In [1]:
from pyomo.environ import *

### Weekly planning, real decision variables

In [2]:
model = ConcreteModel(name="Chess set 1")
model.xs = Var(within=NonNegativeReals)
model.xl = Var(within=NonNegativeReals)
model.obj = Objective(expr= 5*model.xs + 20*model.xl, sense=maximize)
model.latehours = Constraint(expr= 3*model.xs + 2*model.xl <= 160)
model.boxwood = Constraint(expr= 1*model.xs+3*model.xl <= 200)

In [3]:
solver = SolverFactory("glpk")
results = solver.solve(model)
print(results)


Problem: 
- Name: unknown
  Lower bound: 1333.33333333333
  Upper bound: 1333.33333333333
  Number of objectives: 1
  Number of constraints: 3
  Number of variables: 3
  Number of nonzeros: 5
  Sense: maximize
Solver: 
- Status: ok
  Termination condition: optimal
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: 0
      Number of created subproblems: 0
  Error rc: 0
  Time: 0.025753021240234375
Solution: 
- number of solutions: 0
  number of solutions displayed: 0



In [4]:
print(value(model.xs), value(model.xl))

0.0 66.6666666666667


In [5]:
model.obj.pprint()

obj : Size=1, Index=None, Active=True
    Key  : Active : Sense    : Expression
    None :   True : maximize : 5*xs + 20*xl


In [6]:
model.latehours.pprint()

latehours : Size=1, Index=None, Active=True
    Key  : Lower : Body        : Upper : Active
    None :  -Inf : 3*xs + 2*xl : 160.0 :   True


In [7]:
model.boxwood.pprint()

boxwood : Size=1, Index=None, Active=True
    Key  : Lower : Body      : Upper : Active
    None :  -Inf : xs + 3*xl : 200.0 :   True


### Weekly planning, integer decision variables

In [8]:
model = ConcreteModel(name="Chess set 2")
model.xs = Var(within=NonNegativeIntegers)
model.xl = Var(within=NonNegativeIntegers)
model.obj = Objective(expr= 5*model.xs + 20*model.xl, sense=maximize)
model.latehours = Constraint(expr= 3*model.xs + 2*model.xl <= 160)
model.boxwood = Constraint(expr= 1*model.xs+3*model.xl <= 200)

In [9]:
solver = SolverFactory("glpk")
results = solver.solve(model)
print(results)


Problem: 
- Name: unknown
  Lower bound: 1330.0
  Upper bound: 1330.0
  Number of objectives: 1
  Number of constraints: 3
  Number of variables: 3
  Number of nonzeros: 5
  Sense: maximize
Solver: 
- Status: ok
  Termination condition: optimal
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: 1
      Number of created subproblems: 1
  Error rc: 0
  Time: 0.025293588638305664
Solution: 
- number of solutions: 0
  number of solutions displayed: 0



In [10]:
print(value(model.xs), value(model.xl))

2.0 66.0


In [11]:
model.obj.pprint()

obj : Size=1, Index=None, Active=True
    Key  : Active : Sense    : Expression
    None :   True : maximize : 5*xs + 20*xl


In [12]:
model.latehours.pprint()

latehours : Size=1, Index=None, Active=True
    Key  : Lower : Body        : Upper : Active
    None :  -Inf : 3*xs + 2*xl : 160.0 :   True


In [13]:
model.boxwood.pprint()

boxwood : Size=1, Index=None, Active=True
    Key  : Lower : Body      : Upper : Active
    None :  -Inf : xs + 3*xl : 200.0 :   True


### Weekly planning, a generalized approach

In [14]:
def objective(m):
    return 5*m.xs + 20*m.xl

def latehours(m):
    return 3*m.xs + 2*m.xl <= 160

def boxwood(m):
    return 1*m.xs + 3*m.xl <= 200

constraints = [latehours, boxwood]

def constraint(m, idx):
    return constraints[idx](m)

def chess_set(onlyint):
    typ = NonNegativeIntegers if onlyint else NonNegativeReals
    model = ConcreteModel(name="Chess set")
    model.xs = Var(within=typ)
    model.xl = Var(within=typ)
    model.obj = Objective(rule=objective, sense=maximize)
    model.constr = Constraint([0,1], rule=constraint)
    solver = SolverFactory("glpk")
    solver.solve(model)
    return model

In [15]:
res = chess_set(False)
res.pprint()

1 Set Declarations
    constr_index : Dim=0, Dimen=1, Size=2, Domain=None, Ordered=False, Bounds=(0, 1)
        [0, 1]

2 Var Declarations
    xl : Size=1, Index=None
        Key  : Lower : Value            : Upper : Fixed : Stale : Domain
        None :     0 : 66.6666666666667 :  None : False : False : NonNegativeReals
    xs : Size=1, Index=None
        Key  : Lower : Value : Upper : Fixed : Stale : Domain
        None :     0 :   0.0 :  None : False : False : NonNegativeReals

1 Objective Declarations
    obj : Size=1, Index=None, Active=True
        Key  : Active : Sense    : Expression
        None :   True : maximize : 5*xs + 20*xl

1 Constraint Declarations
    constr : Size=2, Index=constr_index, Active=True
        Key : Lower : Body        : Upper : Active
          0 :  -Inf : 3*xs + 2*xl : 160.0 :   True
          1 :  -Inf :   xs + 3*xl : 200.0 :   True

5 Declarations: xs xl obj constr_index constr


In [16]:
res = chess_set(True)
res.pprint()

1 Set Declarations
    constr_index : Dim=0, Dimen=1, Size=2, Domain=None, Ordered=False, Bounds=(0, 1)
        [0, 1]

2 Var Declarations
    xl : Size=1, Index=None
        Key  : Lower : Value : Upper : Fixed : Stale : Domain
        None :     0 :  66.0 :  None : False : False : NonNegativeIntegers
    xs : Size=1, Index=None
        Key  : Lower : Value : Upper : Fixed : Stale : Domain
        None :     0 :   2.0 :  None : False : False : NonNegativeIntegers

1 Objective Declarations
    obj : Size=1, Index=None, Active=True
        Key  : Active : Sense    : Expression
        None :   True : maximize : 5*xs + 20*xl

1 Constraint Declarations
    constr : Size=2, Index=constr_index, Active=True
        Key : Lower : Body        : Upper : Active
          0 :  -Inf : 3*xs + 2*xl : 160.0 :   True
          1 :  -Inf :   xs + 3*xl : 200.0 :   True

5 Declarations: xs xl obj constr_index constr
